In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
movies=pd.read_csv('DATA/imdb_top_1000.csv')

In [3]:
movies.head()

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


In [4]:
movies.dropna(inplace=True)

In [5]:
def convert_to_list(x):
    return x.split(" ")

In [6]:
movies['Genre']=movies['Genre'].apply(convert_to_list)

In [7]:
movies['Overview']=movies['Overview'].apply(convert_to_list)

In [8]:
def remove_spaces(x):
    return x.replace(" ","") 


In [9]:
movies['Star1']=movies['Star1'].apply(remove_spaces)
movies['Star1']=movies['Star1'].apply(convert_to_list)
movies['Star2']=movies['Star2'].apply(remove_spaces)
movies['Star2']=movies['Star2'].apply(convert_to_list)
movies['Star3']=movies['Star3'].apply(remove_spaces)
movies['Star3']=movies['Star3'].apply(convert_to_list)
movies['Star4']=movies['Star4'].apply(remove_spaces)
movies['Star4']=movies['Star4'].apply(convert_to_list)


In [10]:
movies['Director']=movies['Director'].apply(remove_spaces)
movies['Director']=movies['Director'].apply(convert_to_list)

In [11]:
movies['tags']=movies['Genre']+movies['Overview']+movies['Overview']+movies['Director']+movies['Star1']+movies['Star2']+movies['Star3']+movies['Star4']

In [12]:
df=movies[['Series_Title','Released_Year','Certificate','Runtime','Meta_score','tags']]

In [13]:
df.head()

,Series_Title,Released_Year,Certificate,Runtime,Meta_score,tags
0,The Shawshank Redemption,1994,A,142 min,80.0,"[Drama, Two, imprisoned, men, bond, over, a, n..."
1,The Godfather,1972,A,175 min,100.0,"[Crime,, Drama, An, organized, crime, dynasty'..."
2,The Dark Knight,2008,UA,152 min,84.0,"[Action,, Crime,, Drama, When, the, menace, kn..."
3,The Godfather: Part II,1974,A,202 min,90.0,"[Crime,, Drama, The, early, life, and, career,..."
4,12 Angry Men,1957,U,96 min,96.0,"[Crime,, Drama, A, jury, holdout, attempts, to..."


In [14]:
import nltk

In [15]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [16]:
def stem(text):
    my_list=[]
    for i in text:
        my_list.append(ps.stem(i))
    return " ".join(my_list)

In [17]:
df['tags']=df['tags'].apply(stem)

C:\Users\HP\AppData\Local\Temp\ipykernel_8132\4230616273.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags']=df['tags'].apply(stem)


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [19]:
tfidf=TfidfVectorizer(max_features=5000,stop_words='english')

In [20]:
vectors=tfidf.fit_transform(df['tags']).toarray()

In [21]:
vectors

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [22]:
from sklearn.metrics.pairwise import cosine_similarity

In [23]:
similarity=cosine_similarity(vectors)

In [24]:
def recommender(movie):
    movie_index=df[df['Series_Title']==movie].index[0]
    distance=similarity[movie_index]
    movie_list=sorted(list(enumerate(distance)),reverse=True,key=lambda x:x[1])[1:6]

    for i in movie_list:
        print(df.iloc[i[0]].Series_Title)

In [25]:
#The Godfather
recommender('The Dark Knight')

Batman Begins
The Dark Knight Rises
Joker
Kill Bill: Vol. 1
The Hurricane


In [26]:
import pickle

In [27]:
pickle.dump(df.to_dict(),open('movie_dic.pkl','wb'))

In [28]:
pickle.dump(similarity,open('similarity.pkl','wb'))